In [38]:
import json
from tqdm import tqdm

In [30]:
# Load CUHK validate data and targeted LCSH
etd_json = []
with open('./cuhk_etd_abstract_validate.json','r') as f:
    for line in f.readlines():
        line = line.strip("\n")
        if not line:
            continue
        etd_json.append(json.loads(line))
        
etd_result = []
with open('./cuhk_pretrained_wv_result.json','r') as f:
    for line in f.readlines():
        line = line.strip("\n")
        if not line:
            continue
        etd_result.append(json.loads(line))    
        
with open('./enough_train_data_lcsh.json','r') as f:
    enough_lcsh = json.load(f)

### Calculate f1-measure

In [31]:
targeted_lcsh = enough_lcsh.keys()

In [32]:
TP = {i:0.0 for i in targeted_lcsh}
TN = {i:0.0 for i in targeted_lcsh}
FP = {i:0.0 for i in targeted_lcsh}
FN = {i:0.0 for i in targeted_lcsh}

In [41]:
for i in tqdm(targeted_lcsh):
    for true,pred in zip(etd_json,etd_result):
        t = true['lcsh'].keys()
        p = pred.keys()
        if i in t and i in p:
            TP[i] += 1
        if i not in t and i not in p:
            TN[i] += 1
        if i not in t and i in p:
            FP[i] += 1
        if i in t and i not in p:
            FN[i] += 1

100%|██████████| 2928/2928 [00:23<00:00, 124.96it/s]


In [46]:
f_measure = {i:{} for i in targeted_lcsh}

In [48]:
for i in tqdm(targeted_lcsh):
    f_measure[i]['precision'] = TP[i] / (TP[i] + FP[i] + 1e-13)
    f_measure[i]['recall'] = TP[i] / (TP[i] + FN[i] + 1e-13)
    f_measure[i]['f1'] = 2*f_measure[i]['precision']*f_measure[i]['recall'] / (f_measure[i]['precision']+f_measure[i]['recall']+1e-13)

100%|██████████| 2928/2928 [00:00<00:00, 315445.45it/s]


In [54]:
macro_precision = sum(f_measure[i]['precision'] for i in targeted_lcsh)/len(targeted_lcsh)
macro_recall = sum(f_measure[i]['recall'] for i in targeted_lcsh)/len(targeted_lcsh)
macro_f1 = 2*macro_precision*macro_recall/(macro_precision+macro_recall+1e-13)

In [55]:
macro_precision,macro_recall,macro_f1

(0.12668266475820214, 0.1454432223591526, 0.13541626028048218)

### Calculate hit@5

In [65]:
hit_5 = 0.0
cnt = 0
for true,pred in tqdm(zip(etd_json,etd_result)):
    matched = 0
    t = true['lcsh'].keys()
    p = pred.keys()
    if any(elem in t for elem in targeted_lcsh):
        for i in t:
            if i in p:
                matched += 1
        cnt += 1
    hit_5 += matched/len(t)
hit_5 = hit_5 / cnt

10373it [00:01, 9189.86it/s]


In [68]:
hit_5

0.1892589316149126

In [63]:
any(elem in t for elem in targeted_lcsh)

False